In [ ]:
import os.path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error as mse_sklearn
from sklearn.metrics import mean_absolute_error as mae_sklearn

In [ ]:
use_GPU = torch.cuda.is_available()
if use_GPU:
    mode = {"name": "cuda", "device": torch.device("cuda")}
else:
    mode = {"name": "cpu", "device": torch.device("cpu")}

in_dim = 1
hidden_dim = 64
out_dim = 1
sequence_length = 72
batch_size = 128 # 32, 64, 128, 256

num_epochs=10
num_workers=11
lr = 1e-3
regularization=1e-6

In [ ]:
def get_data(path):
    data = []
    for i in ["/train", "/val", "/test"]:
        data.append(pd.read_csv(path + i + ".csv", index_col="Datetime"))
    return data[0], data[1], data[2]
# Load data
path = '../data/clean_data/univariate_Q_Kalltveit'
train, val, test = get_data(path)
train

In [ ]:
test_index = test.index

In [ ]:
# performing preprocessing part
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
  
train_transformed = sc.fit_transform(train)
val_transformed = sc.transform(val)
test_transformed = sc.transform(test)

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y)

In [ ]:
X_train, y_train = sliding_windows(train_transformed, sequence_length)
X_val, y_val = sliding_windows(val_transformed, sequence_length)
X_test, y_test = sliding_windows(test_transformed, sequence_length)

X_train, X_val, X_test = X_train.reshape(len(X_train), -1), X_val.reshape(len(X_val), -1), X_test.reshape(len(X_test), -1)
print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize(mean=(0,0,0),std=(1,1,1))])

In [ ]:
"""# Applying PCA function on training
from sklearn.decomposition import PCA

pca = PCA(n_components = 0.95)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)
sequence_length = 5"""

In [ ]:
from torch.utils.data import Dataset
import torch

class data_trans(Dataset):
        
    def __init__(self, data, groundtruth, transform=None, pca=None):

        self.data = self._get_data(data)
        self.groundtruth = self._get_data(groundtruth)
        self.transform = transform
        self.pca

    def _get_data(self,data):
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
               
        inputs = self.data[index,:]
        groundtruths = self.groundtruth[index,:]
        
        if self.transform:
                        
            inputs = torch.from_numpy(inputs).float()
            groundtruths = torch.from_numpy(groundtruths).float()

        return [inputs, groundtruths]

In [ ]:
train_data_trans = data_trans(X_train, y_train, transform)
val_data_trans = data_trans(X_val, y_val, transform)
test_data_trans = data_trans(X_test, y_test, transform)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data_trans,
                                           batch_size = batch_size,
                                           shuffle = False,
                                           num_workers = 0)
val_dataloader = torch.utils.data.DataLoader(val_data_trans,
                                           batch_size = batch_size,
                                           shuffle = False,
                                           num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(test_data_trans,
                                           batch_size = batch_size,
                                           shuffle = False,
                                           num_workers = 0)

In [ ]:
data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
}

In [ ]:
import torch.nn as nn

class FCN(nn.Module):
    
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length, mode):

        super(FCN, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length
        self.mode = mode

        self.layer_in = nn.Linear(sequence_length, hidden_dim, bias=False)
        self.fcn = nn.Linear(hidden_dim, hidden_dim)
        self.layer_out = nn.Linear(hidden_dim, out_dim, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward (self,input):
        out = self.layer_in(input)
        out = self.sigmoid(out)
        out = self.fcn(out)
        out = self.sigmoid(out)
        out = self.layer_out(out)
      
        return out

In [ ]:
import torch.nn as nn
import torch

class LSTM(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length, mode):
        super(LSTM, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length
        self.mode = mode
        
        # lstm1, lstm2, linear are all layers in the network
        self.lstm1 = nn.LSTMCell(in_dim, hidden_dim)
        self.lstm2 = nn.LSTMCell(hidden_dim, hidden_dim)
        
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.linear_out = nn.Linear(hidden_dim*sequence_length, out_dim)

        
    def forward(self, y):
        outputs = []
        h_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32).to(self.mode["device"])
        c_t = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32).to(self.mode["device"])
        h_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32).to(self.mode["device"])
        c_t2 = torch.zeros(y.size(0), self.hidden_dim, dtype=torch.float32).to(self.mode["device"])
        
        for time_step in y.split(1, dim=1):
            # N, 1
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
        # transform list to tensor    
        outputs = torch.cat(outputs, dim=1)
        out = self.linear_out(outputs)
        return out

In [ ]:
import torch.nn as nn
import torch

class LSTM2(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, sequence_length, mode):
        super(LSTM2, self).__init__()
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.sequence_length = sequence_length
        self.mode = mode

        self.lstm1 = nn.LSTMCell(in_dim, hidden_dim)
        self.lstm2 = nn.LSTMCell(hidden_dim, hidden_dim)

        self.T_A = nn.Linear(sequence_length*hidden_dim, sequence_length)
        
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.linear_out = nn.Linear(hidden_dim, out_dim)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, y):
        outputs = []
        h_t = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        c_t = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        h_t2 = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        c_t2 = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])
        
        for time_step in y.split(1, dim=1):
            h_t, c_t = self.lstm1(time_step, (h_t, c_t)) # initial hidden and cell states
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2)) # new hidden and cell states
            output = self.linear(h_t2) # output from the last FC layer
            outputs.append(output)
            
        total_ht = outputs[0]
        for i in range(1, len(outputs)):
            total_ht = torch.cat((total_ht, outputs[i]), 1)

        beta_t =  self.relu(self.T_A(total_ht))
        beta_t = self.softmax(beta_t)

        out = torch.zeros(y.size(0), self.hidden_dim).to(self.mode["device"])

        for i in range(len(outputs)):
                      
            out = out + outputs[i]*beta_t[:,i].reshape(out.size(0), 1)

        out = self.linear_out(out)
        
        return out

In [ ]:
from tqdm import tqdm
from torchmetrics.functional import mean_absolute_error

def fit(model, loss_function, optimizer, data_loader, num_epochs, mode, use_amp=False):
	history = {"train": {"loss": [], "mae": []}, "val": {"loss": [], "mae": []}}
	scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
	print("\nTraining the model:")
	for epoch in range(num_epochs):
		print("\nEpoch", epoch+1)
		if epoch < num_epochs - 1:
			keys = ["train", "val"]
		else:
			keys = ["train", "val", "test"]
		for key in keys:
			dataset_size = 0
			dataset_loss = 0.0
			#dataset_mae = []
			if key == "train":
				model.train()
			else:
				model.eval()
			for X_batch, y_batch in tqdm(data_loader[key]):
				X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
				with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
					with torch.cuda.amp.autocast(enabled=use_amp): # Mixed-precision support for compatible GPUs
						batch_output = model(X_batch.float())
						batch_loss = loss_function(batch_output, y_batch)
					if key == "train":
						scaler.scale(batch_loss).backward()
						scaler.step(optimizer) 	
						scaler.update()
						optimizer.zero_grad()
				dataset_size += y_batch.shape[0]
				dataset_loss += y_batch.shape[0] * batch_loss.item()
			dataset_loss /= dataset_size
			if key in ["train", "val"]:
				history[key]["loss"].append(dataset_loss)
			else:
				print("\nEvaluating the model:")
			print(key, "loss:", dataset_loss)
	return history


In [ ]:
model_1 = FCN(in_dim, hidden_dim, out_dim, sequence_length).to(mode["device"])
optimizer = optimizer = optim.Adam(model_1.parameters(), lr=lr, weight_decay=regularization)
error_criterion = nn.MSELoss().to(mode["device"])

history = fit(model=model_1, loss_function=error_criterion, optimizer=optimizer, data_loader=data_loader, num_epochs=num_epochs, mode=mode)


In [ ]:
def print_history(history):
	absciss = np.arange(1, len(history["train"]["loss"])+1)
	plt.figure()
	plt.suptitle("Training history")
	plt.subplot(121)
	plt.title("Loss history")
	plt.plot(absciss, history["train"]["loss"], label="Train")
	plt.plot(absciss, history["val"]["loss"], label="Validation")
	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	plt.legend()
	plt.show()
print_history(history)

In [ ]:
model_2 = LSTM(in_dim, hidden_dim, out_dim, sequence_length)
model_2.to(mode["device"])
optimizer = optimizer = optim.Adam(model_2.parameters(), lr=lr, weight_decay=regularization)
error_criterion = nn.MSELoss().to(mode["device"])
history = fit(model=model_2, 
            loss_function=error_criterion, 
            optimizer=optimizer, 
            data_loader=data_loader, 
            num_epochs=num_epochs, 
            mode=mode
            )


In [ ]:
def print_history(history):
	absciss = np.arange(1, len(history["train"]["loss"])+1)
	plt.figure()
	plt.suptitle("Training history")
	plt.subplot(121)
	plt.title("Loss history")
	plt.plot(absciss, history["train"]["loss"], label="Train")
	plt.plot(absciss, history["val"]["loss"], label="Validation")
	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	plt.legend()
	plt.show()
print_history(history)

In [ ]:
model_3 = LSTM(in_dim, hidden_dim, out_dim, sequence_length).to(mode["device"])
optimizer = optimizer = optim.Adam(model_3.parameters(), lr=lr, weight_decay=regularization)
error_criterion = nn.MSELoss().to(mode["device"])
history = fit(model=model_3, loss_function=error_criterion, optimizer=optimizer, data_loader=data_loader, num_epochs=num_epochs, mode=mode)


In [ ]:
def print_history(history):
	absciss = np.arange(1, len(history["train"]["loss"])+1)
	plt.figure()
	plt.suptitle("Training history")
	plt.subplot(121)
	plt.title("Loss history")
	plt.plot(absciss, history["train"]["loss"], label="Train")
	plt.plot(absciss, history["val"]["loss"], label="Validation")
	plt.xlabel("Epoch")
	plt.ylabel("Loss")
	plt.legend()
	plt.show()
print_history(history)

In [ ]:
def predict(model, loader, model_name):
    preds = []
    ground_truth = []
    model.eval()
    df = pd.DataFrame()
    with torch.no_grad():
        for X, y in loader:
            y_pred = model(X.to(mode["device"]))
            preds.append([tensor.item() for tensor in y_pred])
            ground_truth.append([tensor.item() for tensor in y])
    preds = [item for sublist in preds for item in sublist]
    ground_truth = [item for sublist in ground_truth for item in sublist]

    df["pred"] = preds
    df["ground_truth"] = ground_truth
    df.index = test_index[73:]

    mae = mae_sklearn(preds, ground_truth)
    mse = mse_sklearn(preds, ground_truth, squared=True)
    rmse = mse_sklearn(preds, ground_truth, squared=False)

    print(f"Performance measuers with unseen data on {model_name}")
    print("MAE: {:.2f}".format(mae), "MSE: {:.2f}".format(mse), "RMSE: {:.2f}".format(rmse))
    print()

    return df

In [ ]:
model_1_history = predict(model_1, test_dataloader, "FCN")
model_2_history = predict(model_2, test_dataloader, "LSTM")
model_3_history = predict(model_3, test_dataloader, "LSTM Attention")

In [ ]:
import cufflinks as cf
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

model_3_history.iplot()